In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import matplotlib.dates as mdates

### Strategies:

#### 1. Simple Moving Average (MA) Crossover Strategy

A strategy based on the crossover of a short-term and a long-term moving average. The trade signals are:

- **Go Long**: When the short-term moving average crosses above the long-term moving average.
- **Go Short**: When the short-term moving average crosses below the long-term moving average.

---

#### 2. RSI Strategy - Sourced

##### Entry Rules:
- **Go Long** when:
  - Ticker closing price is above its 200-day moving average.
  - The 2-period RSI (Relative Strength Index) of the ticker price closes below 5.

##### Exit Rules:
- **Exit the Trade** when:
  - The ticker price closes above its 5-period moving average.

---

#### 3. Mean Reversion Strategy

A strategy that seeks to capitalize on the tendency of prices to revert back to their average or mean. It assumes that an asset's price will eventually move back toward its historical average after significant deviations.

---


Strategies:
    1 - Simple MA Crossover strategy
    2 - Go long  when: 
            The S&P 500 Index is above its 200-day moving average;
            The 2-period RSI of the S&P 500 Index closes below 5;
        Exit the trade when:
            Exit when the S&P closes above its 5-period moving average.
    3 - Mean Reversion Strategy:
    

In [5]:
class Strategies:
    def __init__(self,df):
        self.df = df.copy()
        
    def Strategy1(self):
        self.df['strategy_1_position'] = np.where(self.df['sma_50'] > self.df['sma_200'], 1, -1)
        return self.df       
    
    def Strategy2(self):
        self.df['strategy_2_position'] = 0
        self.df['strategy_2_position'] = np.where((self.df['close'] < self.df['lower_band']) & (self.df['IBS'] < 0.3),1,
                                       np.where((self.df['close'] > self.df['high'].shift(1)) | (self.df['close'] < self.df['sma_300']),-1,
                                                self.df['position'].shift(1)
                                                )
                                       )
        return self.df
    
    def Strategy3(self):
        self.df['strategy_3_position'] = np.where((self.df['close'] > self.df['sma_200']) & self.df['RSI_2'] < 5, 1,
                                       np.where(self.df['close'] > self.df['sma_5'], -1,
                                                self.df['position'].shift(1)))
        return self.df
    
    
    def compute_return(self):
        self.df[f'strategy_return'] = self.df['position'].shift(1) * self.df['log_ret']
        return self.df 

In [ ]:
plt.figure(figsize=(14,8))

plt.xticks(rotation=45)

plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=3))

plt.plot(final_df.index, final_df['close'], label='AAPL', color='blue')
plt.plot(final_df['sma_50'], label='50 Day SMA', color='orange')
plt.plot(final_df['sma_200'], label='200 Day SMA', color='green')

plt.title('AAPL 50 and 200 Day SMA')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()

# Show the plot
plt.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = final_df.index, y = final_df['close'], mode = 'lines', name = 'AAPL', line = dict(color = 'blue')))
fig.add_trace(go.Scatter(x = final_df.index, y = final_df['sma_50'], mode = 'lines', name = '50 Day SMA', line = dict(color = 'orange')))
fig.add_trace(go.Scatter(x = final_df.index, y = final_df['sma_200'], mode = 'lines', name = '200 Day SMA', line = dict(color = 'green')))
fig.add_trace(go.Scatter)
fig.update_layout(
    title="AAPL 50 and 200 Day SMA",
    xaxis_title="Date",
    yaxis_title="Price",
    template="plotly_dark",
    xaxis=dict(
        tickformat="%Y-%m",
        tickangle=45,
    ),
    width=1000,
    height=600
)

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=final_df.index, y=final_df['close'], mode='lines', name='AAPL', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=final_df.index, y=final_df['sma_50'], mode='lines', name='50 Day SMA', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=final_df.index, y=final_df['sma_200'], mode='lines', name='200 Day SMA', line=dict(color='green')))

# shaded regions for positions
for i in range(1, len(final_df)):
    if final_df['position'].iloc[i] == 1:
        fig.add_shape(type="rect",
                      x0=final_df.index[i-1], y0=min(final_df['close']), 
                      x1=final_df.index[i], y1=max(final_df['close']),
                      fillcolor="green", opacity=0.3, line_width=0)
    elif final_df['position'].iloc[i] == -1:
        fig.add_shape(type="rect",
                      x0=final_df.index[i-1], y0=min(final_df['close']), 
                      x1=final_df.index[i], y1=max(final_df['close']),
                      fillcolor="red", opacity=0.3, line_width=0)


fig.update_layout(
    title="AAPL 50 and 200 Day SMA with Positions",
    xaxis_title="Date",
    yaxis_title="Price",
    template="plotly_dark",
    xaxis=dict(
        tickformat="%Y-%m",
        tickangle=45,
    ),
    width=1000,
    height=600
)

fig.show()
